# Chatbot Using Langchain 

## Introduction

A chat model usually consists of several major components:

* `chat model`: this refers to the chat model interface in LangChain, i.e., the large langauge models.
* `prompt template`: Prompt templates make it easy to assemble prompts that combine default messages, user input, chat history, and (optionally) additional retrieved context.
* `memory`: this refers to the control of memory in the previous conversations.
* `retriever` (optional): These are useful if you want to build a chatbot with domain-specific knowledge.

## Loading Environment Variables

In [1]:
## Set env var OPENAI_API_KEY or load from a .env file:

from dotenv import load_dotenv
load_dotenv('/Users/alvinchen/.env')

True

## Quickstart for a Chatbot

- With a plain chat model, we can get chat completions by passing one or more messages to the model. The chat model will respond with a message.
- The chat model interface is based around messages rather than raw text. The types of messages currently supported in LangChain are `AIMessage`, `HumanMessage`, `SystemMessage`, `FunctionMessage` and `ChatMessage`.

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

chat = ChatOpenAI(model='gpt-4', temperature= 0.6)
chat(
    [
        HumanMessage(
            content="Translate this sentence from English to Chinese: Computational Linguistics is very challenging."
        )
    ]
)

AIMessage(content='计算语言学非常具有挑战性。')

In [4]:
messages = [
    SystemMessage(
        content="You are a professional translator that translates English to Taiwan Mandarin."
    ),
    HumanMessage(content="Computational linguistics is very challenging!"),
]
chat(messages)

AIMessage(content='計算語言學非常具有挑戰性！')

- We can wrap the chat model in a chain, which is designed with built-in memory for remembering the previous converstaional exhcanges.

In [6]:
from langchain.chains import ConversationChain

conversation = ConversationChain(llm=chat)
conversation.run('Translate this sentence from English to Taiwan Mandarin: Computational Linguistics is very challenging.')


'"計算語言學非常具有挑戰性。"'

In [7]:
conversation.run('Support the sentence with two examples.')

'Sure, here are two examples:\n\n1. 在機器翻譯領域，計算語言學非常具有挑戰性。例如，機器需要理解和分析語言的語義與語境，這對於人工智能來說是一項艱鉅的任務。\n\n2. 在語音識別技術中，計算語言學也非常具有挑戰性。例如，機器需要準確識別和轉錄人類的語音，並且需要處理各種方言和口音的差異。'

## Memory

In [8]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("What day is today?")
memory.chat_memory.add_ai_message("Sunday")

In [9]:
memory.load_memory_variables({})

{'history': 'Human: What day is today?\nAI: Sunday'}

- Summary Memory

In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory

llm = ChatOpenAI(temperature=0)


In [27]:

memory = ConversationSummaryMemory(llm=llm)
memory.save_context({"input": "Today is Sunday."}, {"output": "Ok."})
memory.save_context(
    {"input": "Today is November 26, 2023"},
    {"output": "oh, thank you for telling me"},
)

In [29]:
memory.load_memory_variables({})

{'history': 'The human states that today is Sunday and the AI acknowledges it. The human then informs the AI that today is November 26, 2023, and the AI expresses gratitude for the information.'}

- Put memory and llm in the conversation chain

In [30]:
conversation = ConversationChain(llm=llm, memory = memory)

In [31]:
conversation.run("So what date was the day before yesterday?")

'The day before yesterday was November 24, 2023.'

## Conversation Chain

## References

- This tutorial is based on the official documentations, [Chatbots](https://python.langchain.com/docs/use_cases/chatbots).